## Ex 4.1

In [3]:
import time
import tracemalloc
def log_execution (func):
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        start  = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        current, peak = tracemalloc.get_traced_memory()
        print(f"Function: {func.__name__}, Time taken: {end - start} seconds")
        print(f"Function: {func.__name__}, Memory used: {current} bytes, Peak Memory used: {peak} bytes")
        tracemalloc.stop()
        return result
    return wrapper

In [4]:
@log_execution
def find_character (input_string, character):
    for i in range(len(input_string)):
        if input_string[i] == character:
            return True
        else:
            return False

In [5]:
find_character("snfdskfnjsddsjqwieuqweqw", 'a')

Function: find_character, Time taken: 0.0 seconds
Function: find_character, Memory used: 72 bytes, Peak Memory used: 152 bytes


False

## Ex 4.2

In [6]:
def repeat(num_times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = []
            for _ in range(num_times):
                result.append(func(*args, **kwargs))

            return result
        return wrapper
    return decorator

In [7]:
@repeat(num_times=5)
def find_character (input_string, character):
    for i in range(len(input_string)):
        if input_string[i] == character:
            return True
        else:
            return False

In [8]:
find_character("ddfdfsdfsd", "a")

[False, False, False, False, False]

## Ex 4.3

### Not using generator

In [9]:

def read_file_no_yield():
    with open("large_log_file.txt", 'r') as file:
        return file.readlines()

In [10]:
@log_execution
def search_error():
    lines = read_file_no_yield()
    for line in lines:
        if "ERROR" in line:
            pass

In [11]:
search_error()

Function: search_error, Time taken: 16.613926649093628 seconds
Function: search_error, Memory used: 296718 bytes, Peak Memory used: 954409618 bytes


### Using a generator

In [12]:

def read_file_with_yield():
    with open("large_log_file.txt") as file:
        for line in file:
            yield line.strip()
        return file
@log_execution
def search_error():
    for line in read_file_with_yield():
        if "ERROR" in line:
            pass

In [13]:
search_error()

Function: search_error, Time taken: 14.873018026351929 seconds
Function: search_error, Memory used: 149253 bytes, Peak Memory used: 183623 bytes


## Ex 4.4

- MultiThreading: 	Multiple threads within the same process, sharing memory space.
    - Best for: I/O-bound tasks, such as reading/writing files, network requests, and user interfaces where waiting for external resources is common.
- Multiprocessing: Multiple processes with separate memory spaces, each running its own Python interpreter.
    - Best for: CPU-bound tasks, such as mathematical calculations, image processing, and database operations where parallelism is beneficial.
- Asynchronous Programming: Single-threaded event loop managing tasks via cooperative multitasking (tasks yield control).
    - Best for: I/O-bound tasks like handling large numbers of network connections, APIs, real-time systems, and web scraping.

## Ex 4.5

In [14]:
import os
import random
import aiofiles
import shutil

In [15]:
def create_directory(directory = "files"):
    if os.path.exists(directory):
    # If it exists, delete all files in the directory
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove the file
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove the sub-directory
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        # If the directory does not exist, create it
        os.makedirs(directory)

In [16]:
create_directory()

In [17]:
# Step 1: Generate 1000 text files with random numbers of lines
@log_execution
def generate_files(directory = "files"):
    for i in range(1000):
        file_name = f"{directory}/file_{i+1}.txt"
        num_lines = random.randint(1, 500)  # Random number of lines between 1 and 500
        with open(file_name, 'w') as f:
            for _ in range(num_lines):
                f.write("This is a line of text.\n")



In [18]:
generate_files()

Function: generate_files, Time taken: 1.8480300903320312 seconds
Function: generate_files, Memory used: 297267 bytes, Peak Memory used: 339181 bytes


## Ex 4.6

### Not using asynchronisation

In [19]:
# Step 2: Create result.txt to store the number of lines for each file
@log_execution
def create_result_txt(path = "result.txt", directory = "files"):
    with open(path, 'w') as result_file:
        for i in range(1000):
            file_name = f"{directory}/file_{i+1}.txt"
            with open(file_name, 'r') as f:
                lines =  f.readlines() 
                num_lines = len(lines)
            result_file.write(f"{file_name}: {num_lines} lines\n")

In [20]:
create_result_txt()

Function: create_result_txt, Time taken: 5.3719236850738525 seconds
Function: create_result_txt, Memory used: 300384 bytes, Peak Memory used: 414971 bytes


### Using asynchronisation

In [21]:
@log_execution
async def create_result_txt_with_async(path="result.txt", directory="files"):
    async with aiofiles.open(path, 'w') as result_file:
        for i in range(1000):
            file_name = f"{directory}/file_{i+1}.txt"
            async with aiofiles.open(file_name, 'r') as f:
                lines = await f.readlines() 
                num_lines = len(lines) 
            await result_file.write(f"{file_name}: {num_lines} lines\n")

In [22]:
await create_result_txt_with_async()

Function: create_result_txt_with_async, Time taken: 0.0 seconds
Function: create_result_txt_with_async, Memory used: 296 bytes, Peak Memory used: 296 bytes


### Using multi-threading

In [37]:
import threading

# Create a lock for writing to the result file
lock = threading.Lock()
def read_length(file_name):
    with open(file_name, 'r') as f:
        lines =  f.readlines() 
        num_lines = len(lines)
    return num_lines
# Function to write the result for a file to the result.txt
def write_result(file_name, num_lines, result_file):
    with lock:  # Ensure only one thread writes at a time
        result_file.write(f"{file_name}: {num_lines} lines\n")

# Function to process a range of files
def process_files(start, end, directory, result_file):
    for i in range(start, end):
        file_name = f"{directory}/file_{i+1}.txt"
        num_lines = read_length(file_name)
        write_result(file_name, num_lines, result_file)

@log_execution
def create_result_txt(path="result.txt", directory="files", num_threads=5):
    total_files = 1000
    files_per_thread = total_files // num_threads
    threads = []
    # Open the result file in append mode
    with open(path, 'w') as result_file:
        # Create threads
        for i in range(num_threads):
            start = i * files_per_thread
            # Ensure the last thread handles any remaining files
            end = total_files if i == num_threads - 1 else (i + 1) * files_per_thread

            # Create a thread to process a subset of files
            thread = threading.Thread(target=process_files, args=(start, end, directory, result_file))
            threads.append(thread)

        # Start all threads
        for thread in threads:
            thread.start()

        # Wait for all threads to finish
        for thread in threads:
            thread.join()

In [38]:
create_result_txt()

Function: create_result_txt, Time taken: 0.36475372314453125 seconds
Function: create_result_txt, Memory used: 730175 bytes, Peak Memory used: 958948 bytes


In [33]:
import concurrent.futures
import threading

# Create a lock for writing to the result file
lock = threading.Lock()

def read_length(file_name):
    try:
        with open(file_name, 'r') as f:
            lines = f.readlines()
            num_lines = len(lines)
        return num_lines
    except FileNotFoundError:
        return 0  # Handle case where file does not exist

# Function to write the result for a file to the result.txt
def write_result(file_name, num_lines, result_file):
    with lock:  # Ensure only one thread writes at a time
        result_file.write(f"{file_name}: {num_lines} lines\n")

# Function to process a single file
def process_file(i, directory):
    file_name = f"{directory}/file_{i+1}.txt"
    num_lines = read_length(file_name)
    return file_name, num_lines

# Main function using ThreadPoolExecutor
@log_execution
def create_result_txt(path="result.txt", directory="files", num_threads=None):
    total_files = 1000

    # Open the result file in write mode
    with open(path, 'w') as result_file:
        # Use ThreadPoolExecutor to manage threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit all file processing tasks to the thread pool
            futures = [executor.submit(process_file, i, directory) for i in range(total_files)]

            # As each task completes, write the result to the result file
            for future in concurrent.futures.as_completed(futures):
                file_name, num_lines = future.result()  # Get the result of the task
                write_result(file_name, num_lines, result_file)

In [40]:
# Call the main function
create_result_txt()

Function: create_result_txt, Time taken: 0.30861830711364746 seconds
Function: create_result_txt, Memory used: 298649 bytes, Peak Memory used: 529685 bytes


### Using multi-processing

In [26]:
import multiprocessing

# Function to read the number of lines in a file
def read_file(file_name):
    try:
        with open(file_name, 'r') as f:
            lines = f.readlines()
            num_lines = len(lines)
        return file_name, num_lines
    except FileNotFoundError:
        return file_name, 0  # Return 0 if the file does not exist

# Worker function to process a batch of files and return the results
def process_files(file_range, directory, queue):
    results = []
    for i in file_range:
        file_name = f"{directory}/file_{i+1}.txt"
        result = read_file(file_name)
        results.append(result)
    queue.put(results)  # Put the result list into the queue

# Main function to create the result.txt file using multiprocessing
@log_execution
def create_result_txt(path="result.txt", directory="files", num_processes=4):
    total_files = 1000
    files_per_process = total_files // num_processes
    processes = []
    queue = multiprocessing.Queue()

    # Create and start processes
    for i in range(num_processes):
        start = i * files_per_process + 1
        # Ensure the last process handles any remaining files
        end = total_files if i == num_processes - 1 else (i + 1) * files_per_process
        file_range = range(start, end + 1)

        # Create a process to handle a subset of the files
        process = multiprocessing.Process(target=process_files, args=(file_range, directory, queue))
        processes.append(process)
        process.start()

    # Collect results from all processes
    all_results = []
    for _ in range(num_processes):
        all_results.extend(queue.get())  # Get results from the queue

    # Wait for all processes to finish
    for process in processes:
        process.join()

    # Write the results to the result.txt file
    with open(path, 'w') as result_file:
        for file_name, num_lines in all_results:
            result_file.write(f"{file_name}: {num_lines} lines\n")

In [ ]:
create_result_txt()